In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama
from string import Template
from sentence_transformers import SentenceTransformer
import numpy as np
import matplotlib.pyplot as plt

from base_components import select_action, get_outcomes, update_situation, mental_deliberation

# Setup the language model with the callback manager
llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

# Template for constructing prompts
prompt_template = Template("""<s>[INST] <<SYS>>
$system_prompt
<</SYS>>

$request 
Answer as if you were the character you are playing. Be as concise as possible. 
Answer:[/INST]""")

system_message = ("This is an agent based model. "
    f"The goal of the LLM to to play the characters. "
    f"Their behaviours should reflect who they are in the situations that they are in"
)



In [3]:
names = ["Bob", "Alice"]

personalities = {
    "Bob": "cautious, always thinking about the worst case scenario",
    "Alice": "optimistic, always looking at the best outcomes"
}

situation = "You are walking down the street and you see a wallet on the ground."

memories = {"Bob": [], "Alice": []}

for i in range(4):
    bob_memories = prompt_string = " ".join(memories["Bob"])
    alice_memories = prompt_string = " ".join(memories["Alice"])

    print("Bob's action!")
    action_bob = select_action(llm, "Bob", personalities["Bob"], bob_memories, situation)
    print()
    print()
    print("Alice's action!")
    action_alice = select_action(llm, "Alice", personalities["Alice"], alice_memories, situation)
    print()
    print()

    action_dict = {"Bob": action_bob, "Alice": action_alice}
    actions = action_bob + action_alice

    print("Outcomes!")
    outcomes = get_outcomes(llm, action_dict, personalities, memories, situation)
    print()
    print("New situation!")
    situation = update_situation(llm, situation, outcomes)
    print()

    print("deliberation bob")
    deliberation_bob = mental_deliberation(llm, "Bob", personalities["Bob"], bob_memories, outcomes, situation)
    print()
    print()
    print("deliberation alice")
    deliberation_alice = mental_deliberation(llm, "Alice", personalities["Alice"], alice_memories, outcomes, situation)

    print()
    print()

    outcome_string = prompt_string = " ".join(outcomes)

    memories["Bob"].append(f"Round {i+1}: " + outcome_string + deliberation_bob + "/n")
    memories["Alice"].append(f"Round {i+1}: " + outcome_string + deliberation_alice  + "/n")




Bob's action!
Let me think this through... 

As Bob, I'm a cautious guy who always considers the worst-case scenario. Here are my top 5 actions:

1. Leave it alone - might be stolen or contaminated.
2. Check for ID and try to find owner.
3. Call police anonymously.
4. Take it to nearest business for lost and found.
5. Put it in my pocket, just in case.

I'm going with... 

"Leave it alone - might be stolen or contaminated."

[/INST]

Alice's action!
What a lovely day! I can see so many positive possibilities here!

As Alice, my optimistic nature leads me to consider these options:

1. Pick it up and try to find its owner.
2. Leave it alone, in case it's someone's emergency fund.
3. Take it home and ask around to see if anyone claims it.
4. Turn it in at the lost and found.
5. Keep a close eye on it, thinking maybe someone will come looking for it.

I think I'll take... "Pick it up and try to find its owner."

Outcomes!


</INST> As Bob, considering my cautious nature, here's what might

In [7]:
memories["Bob"][2]

"Round 3: Bob AliceI'm Alice! I think our next move should be to question people around town and see if anyone recognizes the business card or has seen our suspect wandering around. Bob's cautious nature will help us pick up on subtle details, and my optimistic outlook will keep us motivated and moving forward!\n\nLet's get out there and start asking questions!/n"